 <a name="top"></a>
# Understanding how medGAN works (with count features)

Author: [Sylvain Combettes](https://github.com/sylvaincom).

Edward Choi's original repository: [medgan](https://github.com/mp2893/medgan). <br/>
My own medGAN repository (that is based on Edward Choi's work): [medgan](https://github.com/sylvaincom/medgan-tips).

The final goal of my project is to use medGAN on my own dataset (electronic health records). For that, I first need to understand how medGAN works. In this notebook, I provide a few code cells and explanations that can help better understand and run medGAN. Because there are some confidentiality issues with the MIMIC-III dataset, I cleared the output of the cells.

Before reading this notebook, be sure to have read [A few additional tips on how to run Edward Choi's medGAN](https://github.com/sylvaincom/medgan-tips/blob/master/01_tips-for-medgan.md).

---
### Tables of Contents

- [Loading the MIMIC-III dataset](#load-mimic)
- [Using process_`mimic.py` and `medgan.py` to generate the fake realistic data](#run)
- [How to interpret `gen-samples.npy`?](#gen-samples)
- [Comparing the (fake) generated samples to the real-life original ones](#comparison)

---
### Imports

In [ ]:
import numpy as np
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt

---
# Loading the MIMIC-III dataset <a name="load-mimic"></a>

## `ADMISSIONS.csv` file

In [ ]:
df_adm = pd.read_csv("ADMISSIONS.csv")
print(df_adm.shape)
df_adm.head()

Do we have a lot of missing values?

In [ ]:
n,p = df_adm.shape
for f in df_adm:
    percentage = sum(df_adm[f].isna())*100/n
    if percentage>0:
        print('Missing values in {}: {}%'.format(f, percentage))

## `DIAGNOSES_ICD.csv` file

In [ ]:
df_ICD = pd.read_csv('DIAGNOSES_ICD.csv')
print(df_ICD.shape)
df_ICD.head()

Do we have a lot of missing values?

In [ ]:
n,p = df_ICD.shape
for f in df_ICD:
    percentage = sum(df_ICD[f].isna())*100/n
    if percentage>0:
        print('Missing values in {}: {}%'.format(f, percentage))

We check if our dataset is balanced. Does one `ICD9_CODE` appear distinctly more than others in proportion?

In [ ]:
df_ICD['ICD9_CODE'].value_counts(normalize=True).head()

---
# Using `process_mimic.py` and `medgan.py` to generate the fake realistic data <a name="run"></a>

This step is detailed in [A few additional tips on how to run Edward Choi's medGAN
](https://github.com/sylvaincom/medgan/blob/master/tips-for-medgan.md).

In short, in the Anaconda prompt, we run:
```
cd C:\Users\<username>\Documents\mimic_count
python process_mimic.py ADMISSIONS.csv DIAGNOSES_ICD.csv training-data "count"
mkdir generated
python medgan.py training-data.matrix ./generated/samples --data_type="count"
python medgan.py training-data.matrix gen-samples --model_file=./generated/samples-999 --generate_data=True --data_type="count"
```
Some default values are `n_epoch=1000` and `n_pretrain_epoch=100`. The computing time took more than 5 hours for me.

From now on, whenever we refer to input or output, we refer to the input and output of `medgan.py` (unless specified otherwise).

---
# How to interpret `gen-samples.npy`? <a name="gen-samples"></a>

We load the `gen-samples.npy` file which is `medgan.py`'s output:

In [ ]:
output = np.load('gen-samples.npy')
df_output = pd.DataFrame(output)
print(df_output.shape)
df_output.head()

What is the range of the values?

In [ ]:
plt.plot(df_output.max(axis=0).tolist(), 'o')
plt.xlabel('Index of feature')
plt.ylabel('Maximum')
plt.show()

Some questions about this data frame:
* What do the columns correspond to? They do not look like `ADMISSIONS.csv` nor `DIAGNOSIS_ICD.csv`.
* What do the rows correspond to?
* Why are is the maximum value of each column inferior to 1?
* Why are the values not count values?

We can find some answers in an issue opened in Edward Choi's GitHub: [How to interpret the samples?](https://github.com/mp2893/medgan/issues/3). In order to understand the output `gen-samples.npy` of `medgan.py`, we are going to back to the input of `medgan.py`: the output of `process_mimic.py`.

Actually, as in the `.matrix` file, each row corresponds to a single synthetic patient and each column corresponds to a specific ICD9 diagnosis code. We can use the `.types` file created by `process_mimic.py` to map each column to a specific ICD9 diagnosis code. Read the beginning part of the source code of `process_mimic.py` for more information about `.types` file:
```python
# Output files
# <output file>.pids: cPickled Python list of unique Patient IDs. Used for intermediate processing
# <output file>.matrix: Numpy float32 matrix. Each row corresponds to a patient. Each column corresponds to a ICD9 diagnosis code.
# <output file>.types: cPickled Python dictionary that maps string diagnosis codes to integer diagnosis codes.
```

What is ICD-9? See [ICD-9](https://en.wikipedia.org/wiki/International_Statistical_Classification_of_Diseases_and_Related_Health_Problems#ICD-9) and [List of ICD-9 codes](https://en.wikipedia.org/wiki/List_of_ICD-9_codes).

We need to round the values ourselves:

In [ ]:
df_output = df_output.round(0)
df_output.head()

We claim that we should delete the rows with missing values (if there are any):

In [ ]:
df_output = pd.DataFrame.dropna(df_output)
print(df_output.shape)

Indeed, in line 406 of `medgan.py`, it is written `nSamples=10000`.

## Understanding the `.types` file (an ouput of `process_mimic.py`)

_cPickled Python dictionary that maps string diagnosis codes to integer diagnosis codes._

In [ ]:
map_dict = pickle.load(open('training-data.types', 'rb'))
print(type(map_dict))
print('An excerpt is:', dict(list(map_dict.items())[0:5]))

Thus, as its name suggests, `process_mimic.py` is really dependent on the MIMIC-III dataset. We probably will not use `process_mimic.py` on our own dataset and only run `medgan.py`. Out of `process_mimic.py`, we only need to understand how the generated `.matrix` file is constructed (lines 109 to 119).

## Understanding the `.pids` file (an ouput of `process_mimic.py`)

_cPickled Python list of unique Patient IDs. Used for intermediate processing_

In [ ]:
id_list = pickle.load(open('training-data.pids', 'rb'))
print(type(id_list))
print('An excerpt is:', id_list[:10])

## Understanding the `.matrix` file (an ouput of `process_mimic.py` and the input of `medgan.py`)

_Numpy float32 matrix. Each row corresponds to a patient. Each column corresponds to a ICD9 diagnosis code._

In [ ]:
input_data_array = pickle.load(open('training-data.matrix', 'rb'))
print(type(input_data_array))
input_data_array

In [ ]:
df_input = pd.DataFrame(input_data_array)
print(df_input.shape)
df_input.head(10)

What is the range of the values?

In [ ]:
plt.plot(df_input.max(axis=0).tolist(), 'o')
plt.xlabel('Index of feature')
plt.ylabel('Maximum')
plt.show()

As we chose, the input data has count values. However, contrary to the input data, the maximum of each variable is not 1. 

We can note that the input of `medgan` and the [output](#gen-samples) of `medgan` have the same number of columns. Thus, `gen-samples.npy` is a (fake) realistic generated dataset corresponding to the `.matrix` file.

---
# Comparing the (fake) generated samples to the real-life original ones  <a name="comparison"></a>

In this section, we wish to compare the accuracy of the (fake) generated dataset considering the original one. Contrary to binary data, we can no longer use dimension-wise probability.

Taking the boxplot of each variable is not efficient for visualisation.

## With statistical indicators: mean, median, variance, quantile

In [ ]:
input_mean = df_input.mean().tolist()
output_mean = df_output.mean().tolist()

start = min(np.min(input_mean), np.min(output_mean))
stop = max(np.max(input_mean), np.max(output_mean))
p = df_input.shape[1]
X = np.linspace(start, stop, num=p+1)

plt.plot(input_mean, output_mean, 'ok', X, X, '-g');

plt.legend(['mean for each feature', 'ideal mean for each feature'])
plt.title('Performance of medGAN on count features')
plt.xlabel('For the real dataset')
plt.ylabel('For the (fake) generated dataset')
plt.show()

The values of the x-axis and the y-axis are ordered.

In [ ]:
input_var = df_input.var().tolist()
output_var = df_output.var().tolist()

start = min(np.min(input_var), np.min(output_var))
stop = max(np.max(input_var), np.max(output_var))
p = df_input.shape[1]
X = np.linspace(start, stop, num=p+1)

plt.plot(input_var, output_var, 'ok', X, X, '-g');

plt.legend(['variance for each feature', 'ideal variance for each feature'])
plt.title('Performance of medGAN on count features')
plt.xlabel('For the real dataset')
plt.ylabel('For the (fake) generated dataset')
plt.show()

In [ ]:
qt = 0.5 # choose quantile between 0 and 1
input_qt = df_input.quantile(qt).tolist()
output_qt = df_output.quantile(qt).tolist()

start = min(np.min(input_qt), np.min(output_qt))
stop = max(np.max(input_qt), np.max(output_qt))
p = df_input.shape[1]
X = np.linspace(start, stop, num=p+1)

plt.plot(input_qt, output_qt, 'ok', X, X, '-g');

plt.legend(['%s quantile for each feature' %(qt), 'ideal %s quantile for each feature' %(qt)])
plt.title('Performance of medGAN on count features')
plt.xlabel('For the real dataset')
plt.ylabel('For the (fake) generated dataset')
plt.show()

## With correlation matrix

In [ ]:
# corr_mat = df_input.corr()

The computing time is too long.

---
Back to [top](#top).